In [62]:
import numpy as np
import math
from IPython.core.debugger import set_trace
import copy

lowerBound, upperBound = -5, 10
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)
bounds = [(lowerBound,upperBound) for i in range(N)]

def f(x):
    #Sphere
    # x in interval [-30, -30]
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock
    #za epsilon = 10e-30 dao nulu
    # x in interval [-30, -30]
    Sum = 0
    for i in range(0, N-1):
        Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    # x in interval [-5, -10]
    Sum1, Sum2 = 0, 0
    for i in range(0,N):
        Sum1 = Sum1 + (x[i])**2
        Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(N)]
u = [upperBound for i in range(N)]

In [63]:
def Delta(x):
    #import pdb
    #set_trace()
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [64]:
from scipy.optimize import minimize

def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

def restartSimplex(x,Lambda):
    N = len(x)
    sim = np.empty([N+1,N])
    sim[0] = x
    for j in range(1,N+1):
        sim[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N)
    

    for i in range(len(sim)):
     
        # Find the minimum element in remaining
        # unsorted array
        min_idx = i
        for j in range(i+1, len(sim)):
            if f(sim[min_idx]) > f(sim[j]):
                min_idx = j

        # Swap the found minimum element with
        # the first element       
        temp = copy.copy(sim[i])
        sim[i] = copy.copy(sim[min_idx])
        sim[min_idx] = copy.copy(temp)
        
    return sim

In [65]:
Nfeval = 1

def callbackFsmall(Xi):
    global Nfeval
    print('small: {0:4d}   {1: 3.6f}'.format(Nfeval, f(Xi)))
    Nfeval += 1

def callbackFbig(Xi):
    global Nfeval
    print('big: {0:4d}   {1: 3.6f}'.format(Nfeval, f(Xi)))
    Nfeval += 1

In [70]:
def DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M, Gamma = 1.5,Beta=-0.5):
    k = 0
    it = 0
    x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    
    
    #set_trace()
    while(k < M):
        #set_trace()
        sim = restartSimplex(x,Lambda)
        temp = minimize(f, x, method='nelder-mead',bounds=bounds, tol=None, callback=None, 
                        options={'maxiter': min(1000,M-k), 'maxfev': None, 'disp': False,
                                 'return_all': False, 'initial_simplex': sim, 'xatol': False, 
                                 'fatol': Epsilon, 'adaptive': False})
        x = copy.copy(temp['x'])
        _k = copy.copy(temp['nfev'])
        s = copy.copy(temp['final_simplex'])
        sim = copy.copy(s[0])
        values = copy.copy(s[1])
        #print(values)
        #set_trace()
        #for i in range(0,N+1):print(f(sim[i]))
        #set_trace()
        
        
        #import pdb
        #pdb.set_trace()
        k = k + _k
        try: xBest
        except NameError: xBest = None
        
        #sim = restartSimplex(x,Lambda)
        if xBest is None or f(x) < f(xBest):
            temp = minimize(f, x, method='nelder-mead',bounds=bounds, tol=None, callback=None, 
                        options={'maxiter': min(M-k,1000), 'maxfev': None, 'disp': False, 
                                 'return_all': False, 'initial_simplex': sim, 'xatol': False, 
                                 'fatol': EpsilonApostrophe, 'adaptive': False})
            xBest = copy.copy(temp['x'])
            _k = copy.copy(temp['nfev'])
            #s = temp['final_simplex']
            #sim = s[0]
            k = k + _k
            print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(xBest))
            
            
        if xBest is not None and k//10000 > it:
            it = k//10000
            print("iteration:", it * 10000, "| best found value: ", f(xBest))
            if(f(x) > 10e-10):x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
        
        
        #set_trace()
        #for i in range(0,N+1):print(f(sim[i]))
        #set_trace()
        
            
        
        values = copy.copy(s[1])
            
        #for i in range(0,N):
        #    sim[i][i] += 10e-5
        
        #same = True
        #for i in range(1,N+1):
        #    if(values[0] - values[i] > 0):
        #        same = False
        #if(same):
            #set_trace()
        #    sim = restartSimplex(sim[0],Lambda)
        
        #for i in range(0,N+1):
        #    values[i] = f(sim[i])
        
        while f(x) < f(sim[-1]):
            #set_trace()
            
            #set_trace()
            #for i in range(0,N+1):print(f(sim[i]))
            #set_trace()
            
            
            sim[-1] = copy.copy(x)
            values[-1] = f(x)
            #set_trace()

            
            for i in range(N,0, -1):
                if(values[i] < values[i-1]):
                    temp1, temp2 = copy.copy(sim[i]), copy.copy(values[i])
                    sim[i], values[i] = copy.copy(sim[i-1]), copy.copy(values[i-1])
                    sim[i-1], values[i-1] = copy.copy(temp1), copy.copy(temp2)
                    

            #print(x)

            x0 = np.zeros(N)
            #set_trace()
            for i in range(1,N+1):
                x0 = x0 + sim[i]
            x0 = x0 / N
            #set_trace()
            x = copy.copy(Gamma*sim[0] + (1-Gamma) * x0)

            sim[0] = copy.copy(x)
            values[0] = f(x)

            for i in range(0,N):
                if(values[i] > values[i+1]):
                    temp1, temp2 = copy.copy(sim[i]), copy.copy(values[i])
                    sim[i], values[i] = copy.copy(sim[i+1]), copy.copy(values[i+1])
                    sim[i+1], values[i+1] = copy.copy(temp1), copy.copy(temp2)
                

        #print(Delta(x))
        
        
        
    return xBest

In [73]:
Lambda = 10
M = 1000000
Epsilon = 10e-5
#set_trace()
EpsilonApostrophe = 0
print(DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M))

***NEW BEST VALUE*** iteration:  4355 | best found value:  8.616073529374951
***NEW BEST VALUE*** iteration:  27401 | best found value:  8.616073529374944
iteration: 20000 | best found value:  8.616073529374944
iteration: 30000 | best found value:  8.616073529374944


/tmp/ipykernel_33021/1738225963.py:11: OptimizeWarning: Initial guess is not within the specified bounds
  temp = minimize(f, x, method='nelder-mead',bounds=bounds, tol=None, callback=None,


***NEW BEST VALUE*** iteration:  39688 | best found value:  2.5799275570299045
***NEW BEST VALUE*** iteration:  52630 | best found value:  1.1551485027098458
iteration: 50000 | best found value:  1.1551485027098458
iteration: 60000 | best found value:  1.1551485027098458
***NEW BEST VALUE*** iteration:  64372 | best found value:  2.1316282072803006e-14
iteration: 70000 | best found value:  2.1316282072803006e-14
iteration: 80000 | best found value:  2.1316282072803006e-14
iteration: 90000 | best found value:  2.1316282072803006e-14
iteration: 100000 | best found value:  2.1316282072803006e-14
iteration: 110000 | best found value:  2.1316282072803006e-14
iteration: 120000 | best found value:  2.1316282072803006e-14
iteration: 130000 | best found value:  2.1316282072803006e-14
iteration: 140000 | best found value:  2.1316282072803006e-14
iteration: 150000 | best found value:  2.1316282072803006e-14
iteration: 160000 | best found value:  2.1316282072803006e-14
iteration: 170000 | best fou

KeyboardInterrupt: 